# Apply Pre-Trained Routing Disagreement Models

This notebook loads pre-trained models and predicts routing disagreement for new O-D pairs.

In [ ]:
# Install dependencies (uncomment if needed)
# !pip install scikit-learn pandas numpy geopandas osmnx geopy earthengine-api census pygris joblib

In [ ]:
# Configuration
DISTANCE_MODEL_PATH = 'models/distance_model.joblib'
TIME_MODEL_PATH = 'models/time_model.joblib'

# API Keys (required for generating features)
CENSUS_API_KEY = 'YOUR_CENSUS_API_KEY'
GEE_PROJECT_ID = 'YOUR_GEE_PROJECT_ID'

# Census geography (update for your study area)
STATE_FIPS = '13'
COUNTY_FIPS = '059'

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point
from geopy.distance import geodesic
from collections import Counter
import joblib
import warnings
warnings.filterwarnings('ignore')

ox.settings.log_console = False
ox.settings.use_cache = True

In [ ]:
# Load models
distance_model = joblib.load(DISTANCE_MODEL_PATH)
time_model = joblib.load(TIME_MODEL_PATH)
print(f"Distance model: {distance_model.n_estimators} trees")
print(f"Time model: {time_model.n_estimators} trees")

In [ ]:
# Load your O-D pairs
df = pd.read_csv('data/sample_od_pairs.csv')

# Column names (update if different)
COL_ORIGIN_LAT = 'origin_lat'
COL_ORIGIN_LON = 'origin_lon'
COL_DEST_LAT = 'dest_lat'
COL_DEST_LON = 'dest_lon'

print(f"Loaded {len(df)} O-D pairs")

## Generate Features

In [ ]:
# Constants
BUFFER_M = 400
CRS_METRIC = 3857

def area_km2(geom):
    return geom.area / 1_000_000

def buffer_from_latlon(lat, lon):
    pt = Point(lon, lat)
    return gpd.GeoSeries([pt], crs=4326).to_crs(CRS_METRIC).buffer(BUFFER_M)[0]

def calculate_haversine(row):
    origin = (row[COL_ORIGIN_LAT], row[COL_ORIGIN_LON])
    dest = (row[COL_DEST_LAT], row[COL_DEST_LON])
    return geodesic(origin, dest).meters

def net_metrics(buf_metric):
    intersect_density = road_len_density = 0.0
    area = area_km2(buf_metric)
    buf_wgs = gpd.GeoSeries([buf_metric], crs=CRS_METRIC).to_crs(4326).iloc[0]
    try:
        G = ox.graph_from_polygon(buf_wgs, network_type="drive_service", simplify=True, retain_all=False)
        if len(G.nodes) and len(G.edges):
            nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)
            edges_m = edges.to_crs(CRS_METRIC)
            road_len_density = edges_m.geometry.length.sum() / area
            nodes_m = nodes.to_crs(CRS_METRIC)
            nodes_cl = gpd.clip(nodes_m, buf_metric)
            if "street_count" in nodes_cl.columns:
                inter_cnt = (nodes_cl["street_count"] >= 3).sum()
            else:
                deg = Counter([u for u, v, k in G.edges(keys=True)] + [v for u, v, k in G.edges(keys=True)])
                inter_cnt = sum(1 for n in nodes_cl.index if deg.get(n, 0) >= 3)
            intersect_density = inter_cnt / area
    except:
        pass
    return intersect_density, road_len_density

In [ ]:
# Feature 1: Straight-line distance
df['Straight_Line_Distance_m'] = df.apply(calculate_haversine, axis=1)

In [ ]:
# Features 2-3: Network metrics
df['Origin_Road_Length_Density_m_km2'] = 0.0
df['Dest_Intersection_Density_n_km2'] = 0.0

for i, row in df.iterrows():
    buf_origin = buffer_from_latlon(row[COL_ORIGIN_LAT], row[COL_ORIGIN_LON])
    _, rd_o = net_metrics(buf_origin)
    df.at[i, 'Origin_Road_Length_Density_m_km2'] = rd_o
    
    buf_dest = buffer_from_latlon(row[COL_DEST_LAT], row[COL_DEST_LON])
    id_d, _ = net_metrics(buf_dest)
    df.at[i, 'Dest_Intersection_Density_n_km2'] = id_d
    
    if (i + 1) % 10 == 0:
        print(f"Processed {i + 1}/{len(df)}")

In [ ]:
# Features 4-5: Topographic data
import ee
ee.Authenticate()
ee.Initialize(project=GEE_PROJECT_ID)

DEM = ee.Image('USGS/SRTMGL1_003')

def get_elevation(lat, lon):
    point = ee.Geometry.Point(lon, lat)
    elev = DEM.sample(point, 30).first().get('elevation').getInfo()
    return elev if elev else 0

df['elev_origin'] = df.apply(lambda r: get_elevation(r[COL_ORIGIN_LAT], r[COL_ORIGIN_LON]), axis=1)
df['elev_dest'] = df.apply(lambda r: get_elevation(r[COL_DEST_LAT], r[COL_DEST_LON]), axis=1)
df['Elevation_Difference_m'] = df['elev_dest'] - df['elev_origin']
df['Slope_Pct'] = (df['Elevation_Difference_m'] / df['Straight_Line_Distance_m']) * 100
df = df.drop(columns=['elev_origin', 'elev_dest'])

In [ ]:
# Feature 6: Population
from census import Census
import pygris

block_groups = pygris.block_groups(state=STATE_FIPS, county=COUNTY_FIPS, year=2020, cache=True)
block_groups = block_groups.to_crs(CRS_METRIC)

c = Census(CENSUS_API_KEY)
pop_data = c.pl.get(('P1_001N',), geo={'for': 'block group:*', 'in': f'state:{STATE_FIPS} county:{COUNTY_FIPS}'}, year=2020)
pop_df = pd.DataFrame(pop_data).rename(columns={'P1_001N': 'Population'})
pop_df['GEOID'] = pop_df['state'] + pop_df['county'] + pop_df['tract'] + pop_df['block group']
pop_df = pop_df[['GEOID', 'Population']].astype({'Population': float})

block_groups = block_groups.merge(pop_df, on='GEOID', how='left')
block_groups['Population'] = block_groups['Population'].fillna(0)

geometry = [Point(lon, lat) for lon, lat in zip(df[COL_ORIGIN_LON], df[COL_ORIGIN_LAT])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=4326).to_crs(CRS_METRIC)
gdf_joined = gdf.sjoin(block_groups[['GEOID', 'Population', 'geometry']], how='left', predicate='within')
df['Population'] = gdf_joined['Population'].fillna(0).values

## Predict

In [ ]:
FEATURE_COLS = [
    'Straight_Line_Distance_m',
    'Origin_Road_Length_Density_m_km2',
    'Dest_Intersection_Density_n_km2',
    'Slope_Pct',
    'Elevation_Difference_m',
    'Population'
]

X = df[FEATURE_COLS]

df['distance_disagreement'] = distance_model.predict(X)
df['time_disagreement'] = time_model.predict(X)
df['distance_disagreement_prob'] = distance_model.predict_proba(X)[:, 1]
df['time_disagreement_prob'] = time_model.predict_proba(X)[:, 1]

print(f"Distance disagreement predicted: {df['distance_disagreement'].sum()}/{len(df)}")
print(f"Time disagreement predicted: {df['time_disagreement'].sum()}/{len(df)}")

In [ ]:
# Export results
df.to_csv('routing_predictions.csv', index=False)